In [1]:
import pandas as pd
import numpy as np

In [2]:
tmp = pd.read_json('media_founders_dump.json')

In [3]:
tmp = tmp[~tmp.inn.isnull()]
tmp['inn'] = tmp['inn'].astype(int).astype(str)
tmp.loc[tmp['inn'].str.len()==9,'inn'] = tmp.loc[tmp['inn'].str.len()==9,'inn'].apply( lambda x: x.zfill(10))
tmp.loc[tmp['inn'].str.len()==11,'inn'] = tmp.loc[tmp['inn'].str.len()==11,'inn'].apply( lambda x: x.zfill(12))

In [4]:
tmp.shape

(30952, 6)

## Sybsidies

In [5]:
subsidies = pd.read_json('founders_subsidies_dump.json')[['dateAgreem','otherRecevers','receiverINN','sumRUR']]

In [6]:
subsidies = subsidies[~subsidies.receiverINN.isnull()]
subsidies['receiverINN'] = subsidies['receiverINN'].astype(int).astype(str)
subsidies.loc[subsidies['receiverINN'].str.len()==9,'receiverINN'] = subsidies.loc[subsidies['receiverINN'].str.len()==9,'receiverINN'].apply( lambda x: x.zfill(10))
subsidies.loc[subsidies['receiverINN'].str.len()==11,'receiverINN'] = subsidies.loc[subsidies['receiverINN'].str.len()==11,'receiverINN'].apply( lambda x: x.zfill(12))

In [7]:
subsidies = subsidies[(subsidies.dateAgreem>='2014-01-01')&(subsidies.dateAgreem<'2017-01-01')]

In [8]:
subsidies['year'] = pd.to_datetime(subsidies.dateAgreem).dt.year

In [9]:
subsidies = subsidies.groupby(['receiverINN','year'])['sumRUR'].sum().reset_index()

In [10]:
subsidies = subsidies.groupby(['receiverINN','year'])['sumRUR'].sum().unstack(level=1).fillna(0).reset_index()

In [11]:
tmp = pd.merge(tmp,subsidies,left_on='inn',right_on='receiverINN',how= 'left')
tmp.drop(['receiverINN','ogrn','okved'],axis=1,inplace=True)

In [12]:
tmp.rename(columns = {2014:'2014_sub',2015:'2015_sub',2016:'2016_sub'},inplace=True)

## Grants

In [92]:
grants = pd.read_json('grants_dump.json')

In [95]:
grants = grants[(grants.filingDate>='2014-01-01')&(grants.filingDate<'2017-01-01')]

In [115]:
grants['year'] = pd.to_datetime(grants.filingDate).dt.year

In [111]:
tmp.loc[~tmp.ogrn.isnull(),'ogrn'] = tmp.loc[~tmp.ogrn.isnull(),'ogrn'].astype(int).astype(str)

In [122]:
grants = grants[grants['OGRN'].isin(tmp['ogrn'])].groupby(['OGRN','year']).sum()['sumRUR'].unstack(level=1).fillna(0)

In [136]:
tmp = pd.merge(tmp,grants,left_on = 'ogrn', right_index = True, how='left').fillna(0)
del tmp['contracts223Sum']

In [138]:
tmp.rename(columns = {2014:'2014_grant',2015:'2015_grant',2016:'2016_grant'},inplace=True)

## Suppliers

In [15]:
supp_df = pd.read_json('suppliers-20171201.json')

In [23]:
# supp_df = supp_df[~supp_df.inn.isnull()]
supp_df['inn'] = supp_df['inn'].astype(str)
supp_df.loc[supp_df['inn'].str.len()==9,'inn'] = supp_df.loc[supp_df['inn'].str.len()==9,'inn'].apply( lambda x: x.zfill(10))
supp_df.loc[supp_df['inn'].str.len()==11,'inn'] = supp_df.loc[supp_df['inn'].str.len()==11,'inn'].apply( lambda x: x.zfill(12))

In [26]:
supp_df = supp_df[supp_df.inn.isin(tmp.inn)]

In [28]:
supp_df = supp_df[['inn','contracts223Sum','contracts223YearStats']]

In [65]:
supp_df.loc[~supp_df['contracts223YearStats'].isnull(),'2014_contracts'] = \
supp_df.loc[~supp_df['contracts223YearStats'].isnull(),'contracts223YearStats'].apply(lambda x: x.get('2014',0))

supp_df.loc[~supp_df['contracts223YearStats'].isnull(),'2015_contracts'] = \
supp_df.loc[~supp_df['contracts223YearStats'].isnull(),'contracts223YearStats'].apply(lambda x: x.get('2015',0))

supp_df.loc[~supp_df['contracts223YearStats'].isnull(),'2016_contracts'] = \
supp_df.loc[~supp_df['contracts223YearStats'].isnull(),'contracts223YearStats'].apply(lambda x: x.get('2016',0))

supp_df.loc[~supp_df['contracts223YearStats'].isnull(),'2017_contracts'] = \
supp_df.loc[~supp_df['contracts223YearStats'].isnull(),'contracts223YearStats'].apply(lambda x: x.get('2017',0))

/Users/slavik/anaconda3/lib/python3.6/site-packages/pandas/core/indexing.py:337: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[key] = _infer_fill_value(value)
/Users/slavik/anaconda3/lib/python3.6/site-packages/pandas/core/indexing.py:517: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


In [68]:
supp_df = supp_df[~supp_df['contracts223YearStats'].isnull()]

In [76]:
def get_summ(dct):
    if type(dct)==dict:
        return dct.get('contractsSum')
    else:
        return 0

In [78]:
supp_df['2014_contracts'] = supp_df['2014_contracts'].apply(get_summ)
supp_df['2015_contracts'] = supp_df['2015_contracts'].apply(get_summ)
supp_df['2016_contracts'] = supp_df['2016_contracts'].apply(get_summ)
supp_df['2017_contracts'] = supp_df['2017_contracts'].apply(get_summ)

/Users/slavik/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
/Users/slavik/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/Users/slavik/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the do

In [86]:
tmp = pd.merge(tmp,supp_df.groupby('inn').sum(),left_on='inn',right_index=True, how='left').fillna(0)

## Total

In [150]:
del tmp['ogrn']

In [151]:
tmp['total_cash'] = tmp.iloc[:,4:].sum(axis=1)

In [153]:
result = tmp.sort_values('total_cash',ascending=False).head(1000)

In [158]:
del result['media']

In [159]:
result.to_csv('sprint_74_result.csv', index = False)

In [173]:
def unpivoting(df, col):
    df['year']=col.split('_')[0]
    df['key']=col.split('_')[1]
    df.rename(columns={col:'value'},inplace=True)
    return df[['name','year','key','value']]

In [167]:
result.rename()

Index(['egrulName', 'inn', 'name', '2014_sub', '2015_sub', '2016_sub',
       '2014_contracts', '2015_contracts', '2016_contracts', '2017_contracts',
       '2014_grant', '2015_grant', '2016_grant', 'total_cash'],
      dtype='object')

In [174]:
lst=[]
for col in ['2014_sub', '2015_sub', '2016_sub',
       '2014_contracts', '2015_contracts', '2016_contracts', '2017_contracts',
       '2014_grant', '2015_grant', '2016_grant']:
    lst.append(unpivoting(result.copy(),col))

In [176]:
pd.concat(lst).to_csv('unpivoted_result.csv',index=False)

In [182]:
result[result['2016_grant']>0]

,egrulName,inn,name,2014_sub,2015_sub,2016_sub,2014_contracts,2015_contracts,2016_contracts,2017_contracts,2014_grant,2015_grant,2016_grant,total_cash
30036,НЕГОСУДАРСТВЕННОЕ ОБРАЗОВАТЕЛЬНОЕ УЧРЕЖДЕНИЕ В...,7816002340,Санкт-Петербургский Гуманитарный университет п...,203741824.0,213887100.0,355666300.0,0.0,0.0,0.0,0.0,3500000.0,8036935.0,4000000.0,788832159.0
650,"ОБЩЕРОССИЙСКАЯ ОБЩЕСТВЕННАЯ ОРГАНИЗАЦИЯ ""СОЮЗ ...",7707035201,"Общероссийская общественная организация ""Союз ...",213816428.0,274171394.0,253302480.0,0.0,1410683.4,1400000.0,609000.0,0.0,0.0,20000000.0,764709985.4
